<a href="https://colab.research.google.com/github/SanjayJanardhan-89/ApacheSparkHandsOn/blob/main/SparkPerfTunin/00_Reading_Query_plans.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup Pyspark


In [1]:
!sudo apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
#Check this site for the latest download link https://www.apache.org/dyn/closer.lua/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!wget -q https://dlcdn.apache.org/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!tar xf spark-3.2.1-bin-hadoop3.2.tgz
!pip install -q findspark
!pip install pyspark
!pip install py4j

import os
import sys
# os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
# os.environ["SPARK_HOME"] = "/content/spark-3.2.1-bin-hadoop3.2"


import findspark
findspark.init()
findspark.find()

import pyspark
from pyspark.sql import SparkSession

spark= SparkSession \
       .builder \
       .appName("OurSparkApp") \
       .getOrCreate()

spark

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:3 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:5 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:6 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:10 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,542 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [3,140 kB]
Get:13 http://security.ubuntu.com/ubuntu jammy-security/universe am

In [2]:
salary_data = [("John", "Field-eng", 3500),
    ("Michael", "Field-eng", 4500),
    ("Robert", None, 4000),
    ("Maria", "Finance", 3500),
    ("John", "Sales", 3000),
    ("Kelly", "Finance", 3500),
    ("Kate", "Finance", 3000),
    ("Martin", None, 3500),
    ("Kiran", "Sales", 2200),
    ("Michael", "Field-eng", 4500)
  ]
columns= ["Employee", "Department", "Salary"]
salary_data = spark.createDataFrame(data = salary_data, schema = columns)
salary_data.printSchema()
salary_data.show()


root
 |-- Employee: string (nullable = true)
 |-- Department: string (nullable = true)
 |-- Salary: long (nullable = true)

+--------+----------+------+
|Employee|Department|Salary|
+--------+----------+------+
|    John| Field-eng|  3500|
| Michael| Field-eng|  4500|
|  Robert|      NULL|  4000|
|   Maria|   Finance|  3500|
|    John|     Sales|  3000|
|   Kelly|   Finance|  3500|
|    Kate|   Finance|  3000|
|  Martin|      NULL|  3500|
|   Kiran|     Sales|  2200|
| Michael| Field-eng|  4500|
+--------+----------+------+



In [5]:
salary_data.rdd.getNumPartitions()

2

In [4]:
salary_data.coalesce(1).explain(True)

== Parsed Logical Plan ==
Repartition 1, false
+- LogicalRDD [Employee#0, Department#1, Salary#2L], false

== Analyzed Logical Plan ==
Employee: string, Department: string, Salary: bigint
Repartition 1, false
+- LogicalRDD [Employee#0, Department#1, Salary#2L], false

== Optimized Logical Plan ==
Repartition 1, false
+- LogicalRDD [Employee#0, Department#1, Salary#2L], false

== Physical Plan ==
Coalesce 1
+- *(1) Scan ExistingRDD[Employee#0,Department#1,Salary#2L]



In [14]:
import requests

# Download CSV file
url = "https://people.sc.fsu.edu/~jburkardt/data/csv/hw_25000.csv"
csv_path = "/tmp/hw_200.csv"

with open(csv_path, "wb") as f:
    f.write(requests.get(url).content)

df = spark.read.option("header", True).csv(csv_path)
df.show(5)

+-----+-----------------+-----------------+
|Index| "Height(Inches)"| "Weight(Pounds)"|
+-----+-----------------+-----------------+
|    1|         65.78331|         112.9925|
|    2|         71.51521|         136.4873|
|    3|         69.39874|         153.0269|
|    4|          68.2166|         142.3354|
|    5|         67.78781|         144.2971|
+-----+-----------------+-----------------+
only showing top 5 rows



In [15]:
df.rdd.getNumPartitions()

1

In [16]:
salary_data.coalesce(1).explain(True)

== Parsed Logical Plan ==
Repartition 1, false
+- LogicalRDD [Employee#0, Department#1, Salary#2L], false

== Analyzed Logical Plan ==
Employee: string, Department: string, Salary: bigint
Repartition 1, false
+- LogicalRDD [Employee#0, Department#1, Salary#2L], false

== Optimized Logical Plan ==
Repartition 1, false
+- LogicalRDD [Employee#0, Department#1, Salary#2L], false

== Physical Plan ==
Coalesce 1
+- *(1) Scan ExistingRDD[Employee#0,Department#1,Salary#2L]



In [32]:
df = spark.read.format("csv").options(header="True").load("/assessments.csv")

In [33]:
df.show()

+-----------+-----------------+-------------+---------------+----+------+
|code_module|code_presentation|id_assessment|assessment_type|date|weight|
+-----------+-----------------+-------------+---------------+----+------+
|        AAA|            2013J|         1752|            TMA|  19|    10|
|        AAA|            2013J|         1753|            TMA|  54|    20|
|        AAA|            2013J|         1754|            TMA| 117|    20|
|        AAA|            2013J|         1755|            TMA| 166|    20|
|        AAA|            2013J|         1756|            TMA| 215|    30|
|        AAA|            2013J|         1757|           Exam|NULL|   100|
|        AAA|            2014J|         1758|            TMA|  19|    10|
|        AAA|            2014J|         1759|            TMA|  54|    20|
|        AAA|            2014J|         1760|            TMA| 117|    20|
|        AAA|            2014J|         1761|            TMA| 166|    20|
|        AAA|            2014J|       

In [34]:
df.rdd.getNumPartitions()

1

In [35]:
df.coalesce(2).explain(True)

== Parsed Logical Plan ==
Repartition 2, false
+- Relation [code_module#292,code_presentation#293,id_assessment#294,assessment_type#295,date#296,weight#297] csv

== Analyzed Logical Plan ==
code_module: string, code_presentation: string, id_assessment: string, assessment_type: string, date: string, weight: string
Repartition 2, false
+- Relation [code_module#292,code_presentation#293,id_assessment#294,assessment_type#295,date#296,weight#297] csv

== Optimized Logical Plan ==
Repartition 2, false
+- Relation [code_module#292,code_presentation#293,id_assessment#294,assessment_type#295,date#296,weight#297] csv

== Physical Plan ==
Coalesce 2
+- FileScan csv [code_module#292,code_presentation#293,id_assessment#294,assessment_type#295,date#296,weight#297] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex(1 paths)[file:/assessments.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<code_module:string,code_presentation:string,id_assessment:string,assessm

In [37]:
df.repartition(10)
df.rdd.getNumPartitions()

1